# Problema de las N-Reinas 2.0

Nuestra función a minimizar será:


$\begin{equation}
f(x) =  -\sum_i^n \sum_j^n x_{i,j} +   \lambda\sum_{i_1}^n \sum_{i_2}^n \sum_{i_3}^n \sum_{i_4}^n J_{i_1,i_2,i_3,i_4}x_{i_1,i_2}x_{i_3,i_4}
\end{equation}$

donde $x_{ij}$ valdrá 1 si en la casilla (i,j) hay una dama y 0 si no.

Daremos además a $J_{i_1,i_2,i_3,i_4}$ el valor 1 si las casillas $(i_1,i_2)$ y $(i_3, i_4)$ están conectadas y 0 en otro caso.

In [16]:
import qubovert

size = 11
lagrange = size ** 2

# Creamos las variables de nuestro modelo
Q = qubovert.QUBO()
for i in range(size):
    for j in range(size):
        Q.create_var(f"x_{i}_{j}")
        
# Añadimos el primer bloque de la función objetivo
for i in range(size):
    for j in range(size):
        Q[(f"x_{i}_{j}",)] = -1
        
# Incluimos las restricciones finales
for i1 in range(size):
    for i2 in range(size):
        for i3 in range(size):
            for i4 in range(size):
                if i1 == i3 or i2 == i4 or i1 - i3 == i2 - i4 or i1 - i3 == i4 - i2:
                    if not (i1 == i3 and i2 == i4):
                        Q[(f"x_{i1}_{i2}", f"x_{i3}_{i4}")] = lagrange
                

In [17]:
Q

{('x_0_0',): -1,
 ('x_0_1',): -1,
 ('x_0_2',): -1,
 ('x_0_3',): -1,
 ('x_0_4',): -1,
 ('x_0_5',): -1,
 ('x_0_6',): -1,
 ('x_0_7',): -1,
 ('x_0_8',): -1,
 ('x_0_9',): -1,
 ('x_0_10',): -1,
 ('x_1_0',): -1,
 ('x_1_1',): -1,
 ('x_1_2',): -1,
 ('x_1_3',): -1,
 ('x_1_4',): -1,
 ('x_1_5',): -1,
 ('x_1_6',): -1,
 ('x_1_7',): -1,
 ('x_1_8',): -1,
 ('x_1_9',): -1,
 ('x_1_10',): -1,
 ('x_2_0',): -1,
 ('x_2_1',): -1,
 ('x_2_2',): -1,
 ('x_2_3',): -1,
 ('x_2_4',): -1,
 ('x_2_5',): -1,
 ('x_2_6',): -1,
 ('x_2_7',): -1,
 ('x_2_8',): -1,
 ('x_2_9',): -1,
 ('x_2_10',): -1,
 ('x_3_0',): -1,
 ('x_3_1',): -1,
 ('x_3_2',): -1,
 ('x_3_3',): -1,
 ('x_3_4',): -1,
 ('x_3_5',): -1,
 ('x_3_6',): -1,
 ('x_3_7',): -1,
 ('x_3_8',): -1,
 ('x_3_9',): -1,
 ('x_3_10',): -1,
 ('x_4_0',): -1,
 ('x_4_1',): -1,
 ('x_4_2',): -1,
 ('x_4_3',): -1,
 ('x_4_4',): -1,
 ('x_4_5',): -1,
 ('x_4_6',): -1,
 ('x_4_7',): -1,
 ('x_4_8',): -1,
 ('x_4_9',): -1,
 ('x_4_10',): -1,
 ('x_5_0',): -1,
 ('x_5_1',): -1,
 ('x_5_2',): -1,
 ('x_5_3'

In [18]:
dwave_dic = {}
for i in Q:
    if len(i) == 1:
        dwave_dic[(i[0],i[0])] = Q[i]
    else:
        dwave_dic[i] = Q[i]
        
dwave_dic

{('x_0_0', 'x_0_0'): -1,
 ('x_0_1', 'x_0_1'): -1,
 ('x_0_2', 'x_0_2'): -1,
 ('x_0_3', 'x_0_3'): -1,
 ('x_0_4', 'x_0_4'): -1,
 ('x_0_5', 'x_0_5'): -1,
 ('x_0_6', 'x_0_6'): -1,
 ('x_0_7', 'x_0_7'): -1,
 ('x_0_8', 'x_0_8'): -1,
 ('x_0_9', 'x_0_9'): -1,
 ('x_0_10', 'x_0_10'): -1,
 ('x_1_0', 'x_1_0'): -1,
 ('x_1_1', 'x_1_1'): -1,
 ('x_1_2', 'x_1_2'): -1,
 ('x_1_3', 'x_1_3'): -1,
 ('x_1_4', 'x_1_4'): -1,
 ('x_1_5', 'x_1_5'): -1,
 ('x_1_6', 'x_1_6'): -1,
 ('x_1_7', 'x_1_7'): -1,
 ('x_1_8', 'x_1_8'): -1,
 ('x_1_9', 'x_1_9'): -1,
 ('x_1_10', 'x_1_10'): -1,
 ('x_2_0', 'x_2_0'): -1,
 ('x_2_1', 'x_2_1'): -1,
 ('x_2_2', 'x_2_2'): -1,
 ('x_2_3', 'x_2_3'): -1,
 ('x_2_4', 'x_2_4'): -1,
 ('x_2_5', 'x_2_5'): -1,
 ('x_2_6', 'x_2_6'): -1,
 ('x_2_7', 'x_2_7'): -1,
 ('x_2_8', 'x_2_8'): -1,
 ('x_2_9', 'x_2_9'): -1,
 ('x_2_10', 'x_2_10'): -1,
 ('x_3_0', 'x_3_0'): -1,
 ('x_3_1', 'x_3_1'): -1,
 ('x_3_2', 'x_3_2'): -1,
 ('x_3_3', 'x_3_3'): -1,
 ('x_3_4', 'x_3_4'): -1,
 ('x_3_5', 'x_3_5'): -1,
 ('x_3_6', 'x_3_6')

In [19]:
from neal import SimulatedAnnealingSampler
#from dwave.system import DWaveSampler, EmbeddingComposite


n_samples = 2000 # número de veces que ejecutamos el sistema


sampler = SimulatedAnnealingSampler()
#sampler = EmbeddingComposite(DWaveSampler())

sampleset = sampler.sample_qubo(dwave_dic, num_reads = n_samples)
solution = sampleset.first.sample


In [20]:
for i in range(size):
    for j in range(size):
        if solution[f"x_{i}_{j}"] == 0:
            print("O", end = " ")
        else:
            print("X", end = " ")
    print()

O O O O O O O O X O O 
O O X O O O O O O O O 
O O O O O O O X O O O 
O O O X O O O O O O O 
O X O O O O O O O O O 
O O O O O O O O O X O 
O O O O X O O O O O O 
O O O O O O X O O O O 
O O O O O O O O O O X 
O O O O O O O O O O O 
O O O O O X O O O O O 
